In [1]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Embedding, LSTM, Input
import matplotlib.pyplot as plt 
from keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
import gensim

Using TensorFlow backend.


In [2]:
import pickle as pk
from io import StringIO
from scipy import spatial
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelBinarizer
from keras.models import Model, Sequential
from keras.layers import LSTM,Dense,Input,Bidirectional, Embedding, Dropout
from keras import regularizers
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle as pk
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import one_hot
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [3]:
df = pd.concat([pd.read_csv('fp.csv'), pd.read_csv('bc.csv'), pd.read_csv('chr.csv')])

In [4]:
df.drop(columns=['Unnamed: 0','factuality'], inplace=True)

In [5]:
df = df[df['polarity'] != 'NOT_LABELED']
df.head()

,text,polarity
0,I have been a coffee drinker on and off since ...,NEUTRAL
1,"I notice now that I am older, and perhaps I ha...",NEGATIVE
2,"When I drink it, if I drink about 2 cups a day...",NEGATIVE
3,I always kind of said oh it's just allergies f...,NEGATIVE
4,"but in fact when I do not consume coffee, I am...",POSITIVE


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text, df.polarity, test_size = 0.1)

In [7]:
bn = LabelBinarizer()
bn.fit(Y_train)
Y_train_bin = bn.transform(Y_train)
Y_test_bin = bn.transform(Y_test)
Y_train_bin

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.text)

In [9]:
word_index = tokenizer.word_index

In [10]:
m = 0
for i in df.text.str.split():
    m = max(m, len(i))

In [11]:
sequences_data = tokenizer.texts_to_sequences(X_train)
x_train = pad_sequences(sequences_data, maxlen= 100)

In [12]:
sequences_data_test = tokenizer.texts_to_sequences(X_test)
x_test = pad_sequences(sequences_data_test, maxlen= 100)

In [13]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/Users/amin/Downloads/w2v_phrase3_npmi.bin')

In [14]:
word2vec.vectors

array([[-0.9376361 , -0.11221236, -0.3269382 , ...,  0.6804032 ,
        -0.89632666, -0.0554571 ],
       [-0.42982864,  0.9497814 , -0.9840495 , ..., -0.99131167,
         0.8676895 , -2.1600645 ],
       [-0.5381607 , -2.2095287 ,  0.21322252, ..., -0.654085  ,
         0.5356211 ,  0.224896  ],
       ...,
       [ 0.01730438,  0.05138063, -0.01850826, ..., -0.12739398,
        -0.07888711, -0.02088662],
       [ 0.04842824,  0.06654533, -0.05844656, ...,  0.01437575,
        -0.01286483, -0.06922842],
       [-0.01075443,  0.10522845,  0.10001759, ...,  0.03036992,
        -0.05260199, -0.13318923]], dtype=float32)

In [15]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 100
N_CLASSES = 3
embedding_dim = 300

In [16]:
from keras.layers import Embedding

nb_words = len(word_index)+1

embedding_matrix = np.random.rand(nb_words, EMBEDDING_DIM)
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

embedding_layer = Embedding(embedding_matrix.shape[0], # or len(word_index) + 1
                            embedding_matrix.shape[1], # or EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Null word embeddings: 0


In [17]:
embedding_matrix.shape

(5335, 300)

In [142]:
from keras.layers import Dense, Input, Flatten, LSTM, Dropout, Bidirectional
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

#filter_sizes = [2,3,4,5,6]
drop = 0.5
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))(embedded_sequences)
lstm_0 = LSTM(activation='tanh',units=50, return_sequences=True)(reshape)
lstm_1 = LSTM(activation='tanh',units=50, return_sequences=True)(lstm_0)
lstm_2 = LSTM(activation='tanh',units=50, dropout=drop)(lstm_1)

dense_0 = Dense(10,activation='tanh')(lstm_2)
out = Dense(N_CLASSES,activation='softmax')(dense_0)

model = Model(inputs=sequence_input, outputs=out)
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 300)          1600500   
_________________________________________________________________
reshape_37 (Reshape)         (None, 100, 300)          0         
_________________________________________________________________
lstm_64 (LSTM)               (None, 100, 50)           70200     
_________________________________________________________________
lstm_65 (LSTM)               (None, 100, 50)           20200     
_________________________________________________________________
lstm_66 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_41 (Dense)             (None, 10)                510       
__________

In [139]:
fp = './weights/lstm_unidir_50/weights_lstm_unidir.hdf'

In [140]:
checkpoint = ModelCheckpoint(fp, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [144]:
model.load_weights(fp)

In [145]:
model.fit(x_train, Y_train_bin, validation_data=[x_test,Y_test_bin], nb_epoch=10, batch_size=16, callbacks=[checkpoint])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 3102 samples, validate on 345 samples
Epoch 1/10
3102/3102 [==============================] - 65s 21ms/step - loss: 0.4807 - acc: 0.8198 - val_loss: 0.9263 - val_acc: 0.6493

Epoch 00001: val_acc improved from 0.64638 to 0.64928, saving model to ./weights/lstm_unidir_50/weights_lstm_unidir.hdf
Epoch 2/10
3102/3102 [==============================] - 71s 23ms/step - loss: 0.3304 - acc: 0.8891 - val_loss: 1.0244 - val_acc: 0.6319

Epoch 00002: val_acc did not improve from 0.64928
Epoch 3/10
 160/3102 [>.............................] - ETA: 1:09 - loss: 0.2093 - acc: 0.9437

KeyboardInterrupt: 

In [146]:
from sklearn.metrics import classification_report

model.load_weights(fp)
y_pred = model.predict(x_test)

output_test = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test_bin, axis=1), output_test))

             precision    recall  f1-score   support

          0       0.68      0.53      0.60       127
          1       0.62      0.78      0.69       154
          2       0.69      0.58      0.63        64

avg / total       0.66      0.65      0.64       345



In [ ]:
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

In [102]:
from keras.layers import Dense, Input, Flatten, LSTM, Dropout, Bidirectional
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D,Conv1D,MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

#filter_sizes = [2,3,4,5,6]
drop = 0.2
num_filters = 32

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))(embedded_sequences)

conv_0 = Conv1D(num_filters, kernel_size=3, padding='valid', kernel_initializer='normal', activation='relu')(reshape)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)

#print(maxpool_0.shape)

#reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))(embedded_sequences)

lstm_0 = Bidirectional(LSTM(activation='tanh',units=40, return_sequences=True))(maxpool_0)
lstm_1 = Bidirectional(LSTM(activation='tanh',units=40, return_sequences=True))(lstm_0)
lstm_2 = Bidirectional(LSTM(activation='tanh',units=40))(lstm_1)

dropout = Dropout(drop)(lstm_2)

dense_0 = Dense(20,activation='tanh')(dropout)
dense_1 = Dense(10,activation='tanh')(dense_0)
out = Dense(N_CLASSES,activation='softmax')(dense_0)

model_cnn_lstm = Model(inputs=sequence_input, outputs=out)
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_cnn_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 100, 300)          1600500   
_________________________________________________________________
reshape_27 (Reshape)         (None, 100, 300)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 98, 32)            28832     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 49, 32)            0         
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 49, 80)            23360     
_________________________________________________________________
bidirectional_35 (Bidirectio (None, 49, 80)            38720     
__________

In [103]:
checkpoint = ModelCheckpoint('./weights/cnn_lstm/weights_cnn_lstm.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [104]:
model_cnn_lstm.fit(x_train, Y_train_bin, validation_data=[x_test,Y_test_bin], nb_epoch=16, batch_size=8, callbacks=[checkpoint])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 2929 samples, validate on 518 samples
Epoch 1/16
2929/2929 [==============================] - 76s 26ms/step - loss: 1.0079 - acc: 0.5186 - val_loss: 0.9916 - val_acc: 0.5425

Epoch 00001: val_acc improved from -inf to 0.54247, saving model to ./weights/cnn_lstm/weights_cnn_lstm.hdf5
Epoch 2/16
2929/2929 [==============================] - 58s 20ms/step - loss: 0.9423 - acc: 0.6019 - val_loss: 1.0241 - val_acc: 0.5116

Epoch 00002: val_acc did not improve from 0.54247
Epoch 3/16
2929/2929 [==============================] - 52s 18ms/step - loss: 0.8992 - acc: 0.6326 - val_loss: 0.9547 - val_acc: 0.5965

Epoch 00003: val_acc improved from 0.54247 to 0.59653, saving model to ./weights/cnn_lstm/weights_cnn_lstm.hdf5
Epoch 4/16
2929/2929 [==============================] - 74s 25ms/step - loss: 0.8649 - acc: 0.6552 - val_loss: 0.9586 - val_acc: 0.5985

Epoch 00004: val_acc improved from 0.59653 to 0.59846, saving model to ./weights/cnn_lstm/weights_cnn_lstm.hdf5
Epoch 5/16
2929/2929 [

KeyboardInterrupt: 

In [69]:
from sklearn.metrics import classification_report

model_cnn.load_weights('./weights/cnn/weights_lstm.hdf5')
y_pred = model_cnn.predict(x_test)

output_test = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test_bin, axis=1), output_test))

             precision    recall  f1-score   support

          0       0.64      0.61      0.62       123
          1       0.66      0.75      0.70       157
          2       0.69      0.54      0.60        65

avg / total       0.66      0.66      0.66       345



In [70]:
print(maxpool_0.shape)

(?, 1, 1, 64)


In [18]:
from keras.layers import Dense, Input, Flatten, LSTM, Dropout, Bidirectional
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D,Conv1D,MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

#filter_sizes = [2,3,4,5,6]
drop = 0.5
num_filters = 32

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))(embedded_sequences)

conv_0 = Conv1D(num_filters, kernel_size=3, padding='valid', kernel_initializer='normal', activation='relu')(reshape)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)

#print(maxpool_0.shape)

#reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))(embedded_sequences)

lstm_0 = Bidirectional(LSTM(activation='tanh',units=40, return_sequences=True))(maxpool_0)
lstm_1 = Bidirectional(LSTM(activation='tanh',units=40, return_sequences=True))(lstm_0)
lstm_2 = Bidirectional(LSTM(activation='tanh',units=40))(lstm_1)

dropout = Dropout(drop)(lstm_2)

dense_0 = Dense(20,activation='tanh')(dropout)
dense_1 = Dense(10,activation='tanh')(dense_0)
out = Dense(N_CLASSES,activation='softmax')(dense_0)

model_cnn_lstm = Model(inputs=sequence_input, outputs=out)
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_cnn_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn_lstm.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          1600500   
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 32)            28832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidi

In [19]:
fp = './weights/cnn_lstm/weights_cnn_lstm-{epoch:02d}-{val_acc:.4f}.hdf'

In [20]:
checkpoint = ModelCheckpoint(fp, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [22]:
model_cnn_lstm.fit(x_train, Y_train_bin, validation_data=[x_test,Y_test_bin], nb_epoch=5, batch_size=16, callbacks=[checkpoint])

Train on 3102 samples, validate on 345 samples
Epoch 1/5
  16/3102 [..............................] - ETA: 33s - loss: 0.2050 - acc: 0.8750

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


3102/3102 [==============================] - 27s 9ms/step - loss: 0.1887 - acc: 0.9420 - val_loss: 1.2690 - val_acc: 0.6580

Epoch 00001: val_acc did not improve from 0.71884
Epoch 2/5
3102/3102 [==============================] - 27s 9ms/step - loss: 0.1165 - acc: 0.9710 - val_loss: 1.3354 - val_acc: 0.6696

Epoch 00002: val_acc did not improve from 0.71884
Epoch 3/5
3102/3102 [==============================] - 28s 9ms/step - loss: 0.1024 - acc: 0.9739 - val_loss: 1.3634 - val_acc: 0.6580

Epoch 00003: val_acc did not improve from 0.71884
Epoch 4/5
3102/3102 [==============================] - 30s 10ms/step - loss: 0.1094 - acc: 0.9668 - val_loss: 1.4331 - val_acc: 0.6783

Epoch 00004: val_acc did not improve from 0.71884
Epoch 5/5
 768/3102 [======>.......................] - ETA: 21s - loss: 0.1003 - acc: 0.9740

KeyboardInterrupt: 

In [23]:
model_cnn_lstm.load_weights('./weights/cnn_lstm/weights_cnn_lstm-10-0.7188.hdf')

In [25]:
from sklearn.metrics import classification_report


y_pred = model_cnn_lstm.predict(x_test)

output_test = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test_bin, axis=1), output_test))

             precision    recall  f1-score   support

          0       0.69      0.67      0.68       109
          1       0.73      0.81      0.77       178
          2       0.72      0.53      0.61        58

avg / total       0.72      0.72      0.72       345



In [18]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

num_filters = 32
filter_sizes = [2,3,4,5]
drop = 0.4
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
# print(embedded_sequences.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedded_sequences)
# print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)
maxpool_3 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[3] + 1, 1), strides=(1,1), padding='valid')(conv_3)


concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,maxpool_3])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units= N_CLASSES, activation='softmax')(dropout)

# this creates a model that includes
model_cnn = Model(inputs=sequence_input, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model_cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1600500     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)    

In [19]:
fp = './weights/cnn/weights_cnn-{epoch:02d}-{val_acc:.4f}.hdf'
checkpoint = ModelCheckpoint(fp, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [22]:
model_cnn.fit(x_train, Y_train_bin, validation_data=[x_test,Y_test_bin], nb_epoch=5, batch_size=16, callbacks=[checkpoint])

Train on 3102 samples, validate on 345 samples
Epoch 1/5
  32/3102 [..............................] - ETA: 18s - loss: 0.3035 - acc: 0.8750

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


3102/3102 [==============================] - 13s 4ms/step - loss: 0.3816 - acc: 0.8572 - val_loss: 0.8036 - val_acc: 0.6464

Epoch 00001: val_acc did not improve from 0.65507
Epoch 2/5
3102/3102 [==============================] - 12s 4ms/step - loss: 0.3660 - acc: 0.8640 - val_loss: 0.8029 - val_acc: 0.6522

Epoch 00002: val_acc did not improve from 0.65507
Epoch 3/5
3102/3102 [==============================] - 12s 4ms/step - loss: 0.3428 - acc: 0.8820 - val_loss: 0.8130 - val_acc: 0.6551

Epoch 00003: val_acc did not improve from 0.65507
Epoch 4/5
3102/3102 [==============================] - 12s 4ms/step - loss: 0.3172 - acc: 0.8901 - val_loss: 0.8072 - val_acc: 0.6551

Epoch 00004: val_acc did not improve from 0.65507
Epoch 5/5
3102/3102 [==============================] - 13s 4ms/step - loss: 0.3047 - acc: 0.8939 - val_loss: 0.8219 - val_acc: 0.6493

Epoch 00005: val_acc did not improve from 0.65507


In [23]:
from sklearn.metrics import classification_report

model_cnn.load_weights('./weights/cnn/weights_cnn-04-0.6551.hdf')
y_pred = model_cnn.predict(x_test)

output_test = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test_bin, axis=1), output_test))

             precision    recall  f1-score   support

          0       0.64      0.59      0.61       123
          1       0.62      0.78      0.69       154
          2       0.83      0.50      0.62        68

avg / total       0.67      0.66      0.65       345

